In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler

from utils import helper, config, rayer, kaggle_dataset_helper
from ml.models.base.brisk_xgboost import BriskXGBoost
from ml.models.base.slug_xgboost import SlugXGBoost
from ml.models.base.slug_ann import SlugANN
from ml.models.base.slug_rf import SlugRF
from ml.models.base.slug_knn import SlugKNN
from ml.models.base.brisk_bagging import BriskBagging


from ml.models import common


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
rayer.get_global_cluster(num_cpus=4)

In [5]:
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-17 09:13:07.094383 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 2 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/5.0 CPU
 0.00/16.764 GiB memory
 0.00/4.898 GiB object_store_memory

Demands:
 {'CPU': 1}: 4+ from request_resources()


In [6]:
df_X, df_y = helper.get_covid_dataset()
df_X = df_X.drop(['location'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33)

ss = StandardScaler()
X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns=df_X.columns)
X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns=df_X.columns)


In [15]:
slug_xgb = SlugXGBoost('slug_xgb', X_train_scalar, X_test_scalar, y_train, y_test, timeout=None)
slug_xgb.boosted_round = 300
slug_xgb.n_trials = 2000

In [16]:
slug_xgb.fetch_model()

2022-11-17T09:16:30CST : INFO : slug_xgboost : __discover_model__ : 154 : Message : slug_xgb: Starting training for trials:2000, boosted rounds: 300, max depth: 10
2022-11-17T09:20:07CST : INFO : slug_xgboost : __discover_model__ : 167 : Message : slug_xgb: Number of trials: 2000
2022-11-17T09:20:07CST : INFO : slug_xgboost : __discover_model__ : 169 : Message : Best trial:1260
2022-11-17T09:20:07CST : INFO : slug_xgboost : __discover_model__ : 171 : Message :   Params: 
2022-11-17T09:20:07CST : INFO : slug_xgboost : __discover_model__ : 173 : Message :     lambda 0.006674042364591673
2022-11-17T09:20:07CST : INFO : slug_xgboost : __discover_model__ : 173 : Message :     alpha 0.1626350027497489
2022-11-17T09:20:07CST : INFO : slug_xgboost : __discover_model__ : 173 : Message :     eta 0.0010000087520772598
2022-11-17T09:20:07CST : INFO : slug_xgboost : __discover_model__ : 173 : Message :     gamma 9.971804888053711
2022-11-17T09:20:07CST : INFO : slug_xgboost : __discover_model__ : 1

In [17]:
slug_xgb.score

[-0.7539621718968121, -0.7707471189746125]

In [30]:
pred = slug_xgb.predict(X)
r2_score(pred, y)


-209433243529.78366

In [25]:
slug_xgb.load_score(threshold=0.1)
slug_xgb.score